# Tutorial 1: Far-field basics

Learn environmental exposure simulations using plane waves.

## What you'll learn

- How far-field simulations work (plane waves from different directions)
- Reading and understanding config files
- Running your first GOLIAT study
- Finding and interpreting results

**Related documentation**: [User guide (far-field workflow)](../user_guide/user_guide.md#far-field-workflow-environmental-exposure)

## Prerequisites and setup

Before starting this tutorial, you need:

### Required software

**Sim4Life 8.2.0**: GOLIAT requires Sim4Life version 8.2.0 with a valid license for the software and the Duke, Ella, Thelonious and Eartha phantoms. Sim4Life is commercial software from [ZMT website](https://zmt.swiss/). Other versions are untested and may not work correctly. Since Sim4Life only runs on Windows, only Windows is supported, although Linux is supported experimentally on Sim4Life web.

**Git Bash**: GOLIAT commands run in a Bash shell. On Windows, use Git Bash (included with Git for Windows). Other shells may still work though but are not tested and not shown in these tutorials.

Install Git from https://git-scm.com/downloads if needed.

### Initial setup

If this is your first time using GOLIAT, run these commands from the repository root:

```bash
cd /path/to/goliat

# 1. Add Sim4Life Python to PATH
source .bashrc

# 2. Install GOLIAT in editable mode
python -m pip install -e .

# 3. Run initialization (downloads models, sets up directories)
goliat init
```

This one-time setup:
- Installs Python dependencies from `requirements.txt`
- Installs GOLIAT package (makes `goliat` command available)
- Downloads phantom models (Duke, Thelonious, Eartha, Ella)
- Downloads antenna CAD models
- Creates required directories (`data/`, `configs/`, `results/`, `logs/`)

The initialization takes 5-10 minutes depending on your internet speed. If you accidentally delete parts of this setup, `goliat init` can repair it. A `.setup_done` file is created under `data/`.

**Already initialized?** Just run `source .bashrc` when opening a new terminal, then skip to the next section.

For detailed setup instructions, see the [quick start guide](../user_guide/quick_start.md).

### Hardware recommendations

**GPU (recommended)**: GOLIAT uses the FDTD solver (iSolve), which runs much faster on GPU.

The solver kernel is configured in `base_config.json`:

```json
"solver_settings": {
  "kernel": "acceleware"  // or "cuda" or "software"
}
```

Use `"acceleware"` if you have an AMD or NVIDIA GPU. Use `"cuda"` for NVIDIA GPUs only. Use `"software"` if you don't have a GPU (much slower).

### Running commands

GOLIAT commands should be run from the repository root directory where you have:

```
goliat/
—— configs/          # Configuration files
—— data/             # Downloaded models
—— goliat/           # Source code
—— results/          # Simulation outputs
—— logs/             # Log files
—— .bashrc           # Shell setup
```

All tutorial commands assume you're in this root directory.

## About the notebook code blocks

This tutorial includes Python code blocks for running commands. These are designed for **Jupyter notebooks** if you prefer that workflow.

If you're running these in VS Code, make sure you set `C:\Program Files\Sim4Life_8.2.0.16876\Python\python.exe` as your VS Code Python (with `>Python: Select interpreter`) and that you have Jupyter IPython installed.

**If you're using notebooks**, run this setup once per session:

In [ ]:
from pathlib import Path
import importlib.util

p = Path.cwd()
while not (p / "scripts" / "notebook_helpers.py").exists():
    p = p.parent
spec = importlib.util.spec_from_file_location("_", p / "scripts" / "notebook_helpers.py")
m = importlib.util.module_from_spec(spec)
spec.loader.exec_module(m)
run_bash = m.get_run_bash()

import IPython

IPython.core.display.max_output_size = None



This helper function lets you run bash commands from Python cells using `run_bash('command')`. The setup also disables output truncation so you can see all command output.

**If you're using bash directly** (recommended), ignore the Python code blocks and just run the commands directly. Make sure to always run `source .bashrc` which adds the Sim4Life python path to your `PATH` first.

---

## Understanding far-field exposure

Far-field simulations model environmental EMF exposure. The RF source is distant from the person (broadcast antennas, base stations, ambient fields).

Instead of placing an antenna near the body, we illuminate the phantom with plane waves from different directions. This builds transfer functions that relate measured E-field strength to absorbed power.

### Key concepts

**Incident directions**

Plane waves can come from 6 orthogonal directions:
- `x_pos`, `x_neg` (front/back)
- `y_pos`, `y_neg` (left/right)
- `z_pos`, `z_neg` (top/bottom)

**Polarizations**

For each direction, the E-field has two orientations:
- `theta` polarization
- `phi` polarization

This tutorial runs 4 simulations (2 directions × 2 polarizations).

![Plane wave directions](../img/tutorials/tut1_s4l_directions.png)

---

## The configuration file

Here's our config for this tutorial. It runs 4 simulations with minimal settings.

In [3]:
run_bash("cat configs/tutorial_1_far_field.json")

Running: source .bashrc && cat configs/tutorial_1_far_field.json

------------------------------------------------------------
{
  "extends": "base_config.json",
  "study_type": "far_field",
  "phantoms": ["thelonious"],
  "frequencies_mhz": [700],
  "far_field_setup": {
    "type": "environmental",
    "environmental": {
      "incident_directions": ["x_pos", "z_neg"],
      "polarizations": ["theta", "phi"]
    }
  },
  "execution_control": {
    "do_setup": true,
    "do_run": true,
    "do_extract": true
  },
  "simulation_parameters": {
    "number_of_point_sensors": 2
  }
}

------------------------------------------------------------

Command completed with return code: 0


0



### Breaking down the config

**Inheritance**

```json
{"extends": "base_config.json"
```

This config builds on `base_config.json`, which has common settings (solver parameters, gridding defaults, convergence criteria). We only override study-specific settings.

The base config includes many parameters you don't need to understand yet. Most will be explained throughout these tutorials as they become relevant. For now, just know it provides sensible defaults for:
- FDTD solver settings (boundary conditions, kernel)
- Convergence detection (auto-termination levels)
- Grid generation (automatic mode, refinement)
- Point sensors for field monitoring
- Execution control flags

You can check it later with `cat configs/base_config.json`, but there's no need to modify it for standard studies.

**Study type**

```json
"study_type": "far_field"
```

Tells GOLIAT this is a far-field study (plane waves), not near-field (antennas).

**Phantom selection**

```json
"phantoms": ["thelonious"]
```

We use Thelonious (male child) for this tutorial. If you don't have a license for Thelonious, feel free to use another phantoms, like Duke.

**Frequency**

```json
"frequencies_mhz": [700]
```

Single frequency at 700 MHz (common cellular band).

**Far-field parameters**

```json
"far_field_setup": {
  "type": "environmental",
  "environmental": {
    "incident_directions": ["x_pos", "z_neg"],
    "polarizations": ["theta", "phi"]
  }
}
```

- Type: "environmental" (standard approach, there's also "auto_induced" for future use)
- Directions: Testing x_pos (front) and z_neg (from below)
- Polarizations: Both theta and phi for each direction
- Total simulations: 2 directions * 2 polarizations = 4 simulations

**Execution control**

```json
"execution_control": {
  "do_setup": true,
  "do_run": true,
  "do_extract": true
}
```

Run all three phases:
1. Setup: Build the Sim4Life scene (phantom, plane wave, grid)
2. Run: Execute the EM simulation
3. Extract: Pull SAR data from outputs

For full config reference: [configuration guide](../developer_guide/configuration.md)

![Screenshot of the config file](../img/tutorials/tut1_config.png)

---

## Running the study

This will launch the GUI, run 4 simulations (a few minutes each depending on hardware), and extract SAR results.

In [ ]:
run_bash("goliat study tutorial_1_far_field")

Running: source .bashrc && goliat study tutorial_1_far_field

------------------------------------------------------------
Starting Sim4Life application... 
Initializing Application [stdout]
Initializing Application [stderr]
[Warn]  Unable to load module 'C:\Program Files\Sim4Life_8.2.0.16876\MusaikInterface.xdll'
Josua    : [Info]  Sync
Josua    : [Info]  Sync
Josua    : [Info]  Command [Query Handshake] <ba838023-b536-461a-9e49-4d8334c0db09;127.0.0.1;WIN10-NEW>
Josua    : [Info]  Property [CAresSettings]
[Info]  Connection to local Ares successfully established.
Sim4Life application started. 
--- Starting Far-Field Study: tutorial_1_far_field.json --- [FarFieldStudy._run_study]

--- Processing Simulation 1/4: thelonious, 700MHz, x_pos, theta --- [FarFieldStudy._run_study]
--- Starting: setup --- [profile]
Project path set to: C:/Users/user/repo-clean/results/far_field/thelonious/700MHz/environmental_theta_x_pos/far_field_thelonious_700MHz_environmental_theta_x_pos.smash [ProjectManag

0



### What happened

The GOLIAT GUI opened and ran everything automatically, while your Bash shell logged a more verbose output, including the one from Sim4Life.

![GOLIAT GUI](../img/tutorials/tut1_gui.gif)

You can click around the various tab which are explained in another tutorial. The 'time remaining' is computed based on live timing results and improves as profiling data becomes available in your study.

**Phase breakdown**

**Setup phase:**
    - Created `.smash` project in Sim4Life
    - Placed phantom in scene
    - Configured plane wave source (E-field = 1 V/m)
    - Set up computational grid and various settings
    - Assigned tissue materials from IT'IS database
    - Voxelized the grid and saved the project.

**Run phase:**
    - Executed FDTD solver via iSolve using AXWare GPU acceleration
    - Monitored convergence
    - Saved EM fields to `*_Output.h5`

**Extract phase:**
    - Calculated whole-body average SAR
    - Extracted tissue-specific SAR values
    - Computed peak spatial-average SAR (psSAR10g)
    - Generated JSON, pickle, and HTML reports

This happened 4 times (once per direction/polarization).

---

## What happened in Sim4Life

GOLIAT automatically built scenes in Sim4Life while running.

![Thelonious phantom setup in Sim4Life GUI](../img/tutorials/tut1_setup.png)
![Thelonious phantom with plane wave](../img/tutorials/tut1_result.png)

### Direction

Each incident direction creates different exposure:
- **x_pos**: Wave travels in +X direction (front of phantom)
- **z_neg**: Wave travels in -Z direction (from below)

### Polarization

For each direction, the E-field orientation differs:
- **theta**: E-field in one plane
- **phi**: E-field perpendicular to theta

---

## Exploring results

GOLIAT organizes results in a structured directory tree.

In [ ]:
run_bash("ls -R results/far_field/thelonious/700MHz/")

Running: source .bashrc && ls -R results/far_field/thelonious/700MHz/

------------------------------------------------------------
results/far_field/thelonious/700MHz/:
environmental_phi_x_pos
environmental_phi_z_neg
environmental_theta_x_pos
environmental_theta_z_neg

results/far_field/thelonious/700MHz/environmental_phi_x_pos:
config.json
far_field_thelonious_700MHz_environmental_phi_x_pos.smash
far_field_thelonious_700MHz_environmental_phi_x_pos.smash_Results
point_sensor_data.png
progress.log
sar_results.json
sar_stats_all_tissues.html
sar_stats_all_tissues.pkl
verbose.log

results/far_field/thelonious/700MHz/environmental_phi_x_pos/far_field_thelonious_700MHz_environmental_phi_x_pos.smash_Results:
04895e5e-6bf5-4bd1-8807-2fa8361581a6_AxLog.log
04895e5e-6bf5-4bd1-8807-2fa8361581a6_AxLog.logAxSimSummary1.html
04895e5e-6bf5-4bd1-8807-2fa8361581a6_Input.h5
04895e5e-6bf5-4bd1-8807-2fa8361581a6_Input.log
04895e5e-6bf5-4bd1-8807-2fa8361581a6_Output.h5
04895e5e-6bf5-4bd1-8807-2fa8361581a

0



You'll see 4 directories (one per simulation):

```
results/far_field/thelonious/700MHz/
├── environmental_theta_x_pos/
├── environmental_phi_x_pos/
├── environmental_theta_z_neg/
└── environmental_phi_z_neg/
```

**Note**: Directory names use the format `environmental_{polarization}_{direction}` (e.g., `theta_x_pos` means theta polarization, x_pos direction).

Each contains:
- `*.smash` (Sim4Life project file)
- `*.smash_Results/*_Input.h5` (simulation input settings file)
- `*.smash_Results/*_Output.h5` (simulation results with EM fields)
- `sar_results.json` (extracted SAR values)
- `sar_stats_all_tissues.pkl` (detailed tissue data)
- `sar_stats_all_tissues.html` (HTML report of tissue SAR)
- `point_sensor_data.png` (field convergence plot)
- `config.json` (metadata with config hash, completion status)
- `verbose.log` a log file with everything printed (including Sim4Life)
- `progress.log` a log file with only high-level progress prints (as seen in the GUI also)

Note that the two log files are *for that specific simulation*, and if you want to see the full log file, you can find it under `logs/*.log` with a timestamp as filename.

![Results tree](../img/tutorials/tut1_results_tree.png)

### Looking at SAR results

Check the JSON from one simulation:

In [ ]:
run_bash("cat results/far_field/thelonious/700MHz/environmental_theta_x_pos/sar_results.json")

Running: source .bashrc && cat results/far_field/thelonious/700MHz/environmental_theta_x_pos/sar_results.json

------------------------------------------------------------
{
    "input_power_W": 0.0005309910780076142,
    "input_power_frequency_MHz": 700.0,
    "eyes_group_weighted_avg_sar": 1.282829580112987e-05,
    "eyes_group_peak_sar": 1.6668154785293154e-05,
    "skin_group_weighted_avg_sar": 1.734516314368961e-05,
    "skin_group_peak_sar": 0.0001515590847702697,
    "brain_group_weighted_avg_sar": 1.109098380334678e-05,
    "brain_group_peak_sar": 3.142984496662393e-05,
    "genitals_group_weighted_avg_sar": 3.8027582534518573e-06,
    "genitals_group_peak_sar": 7.225423360068817e-06,
    "whole_body_sar": 1.0254515500239604e-05,
    "peak_sar_10g_W_kg": 0.00015270481526385993,
    "peak_sar_details": {
        "PeakValue": 0.00015270481526385993,
        "PeakLocation": [
            -0.13716261088848114,
            0.0010393965058028698,
            -0.18749523162841797
    

0


**Key metrics**

- **Whole-body SAR**: Average absorption across entire phantom (here 1.02e-5 mW/kg per 1W input power)
- **SAR of groups**: The weighted average SAR for a number of groups (brain, genitals, skin, eyes), which are defined under `data/material_name_mapping.json`
- **Peak 10g SAR**: Maximum SAR averaged over 10g tissue cube (as defined in the 62704-1 IEC/IEEE standard), in both the whole body as in the groups
- **Power balance**: Energy conservation check (should be close to 100%). Note that the input power comes from a theoretical calculation as Sim4Life does not (yet) support input power of a far-field source.

All values are normalized to 1W input power. Scale them based on actual exposure levels.

### HTML report

Open `sar_stats_all_tissues.html` in a browser. It shows a detailed table of SAR values for every tissue type present in the phantom, the group data, and more details 

![HTML report](../img/tutorials/tut1_html_report.png)

This helps identify which tissues absorb the most energy.

### Point sensor convergence plots

Each simulation includes a plot showing E-field magnitude over time at monitoring points.

![Point sensor plot](../img/tutorials/tut1_point_sensor_plot.png)

These plots verify that the simulation ran long enough. You should see:
- Initial oscillations as fields build up
- Gradual decay as steady state is reached
- Stable final values (convergence)

If fields are still changing rapidly at the end, the simulation may need more time. You can look in the detailed logs to examine if `iSolve` detected convergence for a specific point sensor.

### Simulation time

The simulation time is controlled primarily by the following settings:

```json
"simulation_parameters": {
  "global_auto_termination": "GlobalAutoTerminationUserDefined",
  "convergence_level_dB": -15,
  "simulation_time_multiplier": 3.5,
  "number_of_point_sensors": 2
}
```

**Key parameters**:

- `global_auto_termination`: As defined in the Sim4Life manual, one can pick how stringent the convergence detection algo is.
- `convergence_level_dB`: For user-defined termination, you can control the field decay threshold here.
- `simulation_time_multiplier`: This controls the total time the simulation can maximally take. It starts from the theoretical time to cross the bounding box's diagonal at the speed of light in a vacuum, then multiplies this amount by the multiplier.
- `number_of_point_sensors`: How many monitoring points to place (default 2). You can control where these are placed with `point_source_order` and pick up to 8 points (for each corner).

Sim4Life includes built-in convergence detection that automatically stops simulations when fields reach steady state. This prevents wasting computation time after the solution has converged.

Sim4Life offers several preset termination criteria:

- `GlobalAutoTerminationNone`: No automatic termination (runs full specified time)
- `GlobalAutoTerminationWeak`
- `GlobalAutoTerminationMedium`
- `GlobalAutoTerminationStrict`
- `GlobalAutoTerminationUserDefined`: Custom threshold (what we used)

The details can be found in the Sim4Life manual. The field sensor convergence as a whole is taken into account.

**GOLIAT configuration**:

The base config uses `GlobalAutoTerminationUserDefined` with a -15 dB threshold:

```json
"global_auto_termination": "GlobalAutoTerminationUserDefined",
"convergence_level_dB": -15
```

This means the simulation stops when transient energy decays to 1/1000 (10^(-15/10)) of its initial value, which provides sufficient accuracy for SAR studies.

---

## What you learned

1. Far-field studies use plane waves to model environmental exposure
2. Config files control everything (phantoms, frequencies, directions, polarizations)
3. GOLIAT automates the entire workflow (setup, run, extract)
4. Results are organized by study type, phantom, frequency, and scenario
5. SAR metrics quantify absorption (whole-body, localized, peak 10g)

### Try this

Edit `tutorial_1_far_field.json` and:
- Add more directions (`y_pos` and `y_neg`)
- Add another phantom (`eartha` for adult female)
- Add another frequency (`835`)

Then rerun:

In [ ]:
run_bash("goliat study tutorial_1_far_field.json")



Watch the GUI progress through more simulations.

---

## Next steps

- **Tutorial 2**: Configuration and caching (how configs work, avoiding redundant runs)
- **Configuration guide**: Full parameter reference at [../developer_guide/configuration.md](../developer_guide/configuration.md)
- **User guide**: Far-field workflows at [user_guide.md](../user_guide/user_guide.md#far-field-workflow-environmental-exposure)
- **Features**: See what else GOLIAT does at [Full List of Features](../reference/full_features_list.md)

Ready for tutorial 2? Learn how configs extend each other and how GOLIAT avoids redundant work.